In [ ]:
import tensorflow as tf
import traitlets
from math import abs, pi
from jetbot import Robot
from dataset_builder import Camera
from data_preprocess import process_image_for_model


def denormalize_vector(vector: float) -> float:
    """
    Denormalize the normalized vector angle so the robot can use it again.
    :param vector: normalized vector value between 0.0 and 1.0
    :return: vector value between 0.25 and 0.75
    """
    return (vector * 0.5) + 0.25


class AutoPilot(traitlets.HasTraits):
    def __init__(self):
        self.robot: Robot = Robot()
        print('')
        self.camera: Camera = Camera.instance()
        print('Camera initialized')
        self.image_latest = widgets.Image(format='png', width=IMAGE_WIDTH, height=IMAGE_HEIGHT)
        self.controls = {'left_motor': MotorValues(),
                         'right_motor': MotorValues()}

        self.driving_model: tf.keras.models.Model = tf.keras.models.load_model('driving_model.model')

        # Dynamically link motor controller values to class attribute
        self.left_link = traitlets.dlink((self.controls['left_motor'], 'motor_speed'), (self.robot.left_motor, 'value'))
        self.right_link = traitlets.dlink((self.controls['right_motor'], 'motor_speed'), (self.robot.right_motor, 'value'))
        
        # Dynamically link camera output to class attribute
        self.camera_link = traitlets.dlink((self.camera, 'value'), (self.image_latest, 'value'), transform=lambda value: bytes(cv2.imencode('.png', value)[1]))
        
        # Display latest frame from camera
        display(self.image_latest)
    
    def shutdown(self):
        print('Shutdown procedure initiated...')
        self.left_link.unlink()
        self.right_link.unlink()
        self.camera_link.unlink()
        self.camera.stop()
        self.robot.stop()
        print('Shutdown complete')
        exit(0)

    def drive(self):
        while True:
            prediction = self.driving_model.predict([process_image_for_model(self.image_latest.value)])
            steering_rads = denormalize_vector(prediction[0]) * pi

            x = cos(remap_rads)
            y = sin(remap_rads)

            # Calculate intermediaries for tank control conversion
            v = (1 - abs(x)) * (y / 1) + y
            w = (1 - abs(y)) * (x / 1) + x

            # Translate intermediaries and apply right trigger speed multiplier to compute tank control values
            self.controls['left_motor'].motor_speed = (v + w) / 2 * 0.5
            self.controls['right_motor'].motor_speed = (v - w) / 2 * 0.5

if __name__ == '__main__':
    try:
        auto_pilot = AutoPilot()
    except Exception as e:
        auto_pilot.shutdown()
        print(e)
    
    auto_pilot.drive()
    auto_pilot.shutdown()